In [1]:
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import UpSampling2D
from keras.datasets import mnist
from keras.utils import *
import matplotlib.pyplot as plt
from keras import optimizers
from keras import losses
import numpy as np
import random


Using TensorFlow backend.


In [2]:
# Generate training set
batch_size = 1280
gt_all = os.listdir('gt')

batch_train_x = np.zeros((batch_size,64,64,3))
batch_train_y = np.zeros((batch_size,64,64,3))
for a in range(batch_size):
    i = random.randint(0,len(gt_all)-1)

    name = gt_all[i]
    name = name.replace('_segmentation','')
    name = name.replace('png','jpg')
    img = cv2.imread(os.path.join('gt',gt_all[i]))
    img = cv2.resize(img,(64,64))
    batch_train_y[a] = img
    if name in os.listdir('melanoma'):
        img = cv2.imread(os.path.join('melanoma',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_train_x[a] = img
    else:
        img = cv2.imread(os.path.join('others',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_train_x[a] = img
    
print('Done')

Done


In [3]:
#Generate test set
batch_size_test = 300
batch_test_x = np.zeros((batch_size_test,64,64,3))
batch_test_y = np.zeros((batch_size_test,64,64,3))
for a in range(batch_size_test):
    i = random.randint(0,len(gt_all)-1)

    name = gt_all[i]
    name = name.replace('_segmentation','')
    name = name.replace('png','jpg')
    img = cv2.imread(os.path.join('gt',gt_all[i]))
    img = cv2.resize(img,(64,64))
    batch_test_y[a] = img
    if name in os.listdir('melanoma'):
        img = cv2.imread(os.path.join('melanoma',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_test_x[a] = img
    else:
        img = cv2.imread(os.path.join('others',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_test_x[a] = img
            
print('Done')

Done


In [4]:
#Generate Validation set
batch_size_val = 600
batch_val_x = np.zeros((batch_size_val,64,64,3))
batch_val_y = np.zeros((batch_size_val,64,64,3))
for a in range(batch_size_val):
    i = random.randint(0,len(gt_all)-1)

    name = gt_all[i]
    name = name.replace('_segmentation','')
    name = name.replace('png','jpg')
    img = cv2.imread(os.path.join('gt',gt_all[i]))
    img = cv2.resize(img,(64,64))
    batch_val_y[a] = img
    if name in os.listdir('melanoma'):
        img = cv2.imread(os.path.join('melanoma',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_val_x[a] = img
    else:
        img = cv2.imread(os.path.join('others',name))
        if img is not None:
            img = cv2.resize(img,(64,64))
            batch_val_x[a] = img
            
print('Done')

Done


In [24]:
# CNN model with 4 layers of convolution (32 filters), Maxpool (2,2), 4 layers of convolution (64 filters), Maxpool(2,2), 5 layers of convolution (32 filters), Upsampling to image size, one layer fo convolution (3 filters)
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),padding='same',
                 activation='relu',input_shape = (64,64,3)))
model2.add(Conv2D(32, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(32, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(32, (3,3),padding='same',activation='tanh'))

model2.add(MaxPooling2D(pool_size = (2,2),strides=(2,2)))
model2.add(Conv2D(64, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(64, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(64, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(64, (3,3),padding='same',activation='tanh'))

model2.add(MaxPooling2D(pool_size = (2,2),strides=(2,2)))
model2.add(Conv2D(32, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(32, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(32, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(32, (3,3),padding='same',activation='tanh'))
model2.add(Conv2D(32, (3,3),padding='same',activation='tanh'))

model2.add(UpSampling2D(size = (4,4)))
model2.add(Conv2D(3, (3,3),padding='same',activation='relu'))

In [25]:
model2.compile(loss=losses.binary_crossentropy,
              optimizer=optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [26]:
model2.fit(batch_train_x,batch_train_y,validation_data=(batch_val_x,batch_val_y),batch_size =50,epochs=10)

Train on 1280 samples, validate on 600 samples
Epoch 1/10
1280/1280 [==============================] - 190s 149ms/step - loss: 604.4231 - acc: 0.7128 - val_loss: 792.1692 - val_acc: 0.8060
Epoch 2/10
1280/1280 [==============================] - 190s 148ms/step - loss: 805.2259 - acc: 0.8028 - val_loss: 792.1692 - val_acc: 0.8060
Epoch 3/10
1280/1280 [==============================] - 198s 155ms/step - loss: 805.2258 - acc: 0.8028 - val_loss: 792.1692 - val_acc: 0.8060
Epoch 4/10
1280/1280 [==============================] - 231s 181ms/step - loss: 805.2259 - acc: 0.8028 - val_loss: 792.1692 - val_acc: 0.8060
Epoch 5/10
1280/1280 [==============================] - 238s 186ms/step - loss: 805.2259 - acc: 0.8028 - val_loss: 792.1692 - val_acc: 0.8060
Epoch 6/10
1280/1280 [==============================] - 249s 194ms/step - loss: 805.2259 - acc: 0.8028 - val_loss: 792.1692 - val_acc: 0.8060
Epoch 7/10
1280/1280 [==============================] - 248s 194ms/step - loss: 805.2258 - acc: 0.802

In [27]:
model2.evaluate(batch_test_x,batch_test_y,verbose=1)

300/300 [==============================] - 18s 59ms/step


[794.24844441731773, 0.80541178464889529]